In [1]:
from seamless.highlevel import Context, Transformer
import numpy as np

ctx = Context()

In [2]:
ctx.pdb1 = open("1AKE-flanks.pdb").read()
ctx.pdb2 = open("1AKE-B-hit.pdb").read()
ctx.load_pdb1 = Transformer()
ctx.load_pdb1.pdb = ctx.pdb1
ctx.load_pdb_code >> ctx.load_pdb1.code
ctx.load_pdb_code.mount("load_pdb.py")
ctx.flanks = ctx.load_pdb1

ctx.load_pdb2 = Transformer()
ctx.load_pdb2.pdb = ctx.pdb2
ctx.load_pdb2.code = ctx.load_pdb_code
ctx.dbca = ctx.load_pdb2

ctx.get_flank1 = lambda flanks: flanks[:4]
ctx.get_flank1.flanks = ctx.flanks
ctx.flank1 = ctx.get_flank1

ctx.get_flank2 = lambda flanks: flanks[-4:]
ctx.get_flank2.flanks = ctx.flanks
ctx.flank2 = ctx.get_flank2

ctx.equilibrate()
print(ctx.flank1.value)
print(ctx.flank2.value)

Execute local
Execute local
Execute local
Waiting for: [Seamless transformer: .get_flank1.tf, Seamless transformer: .get_flank2.tf]
Execute local
[[26.3220005  54.8409996  13.68799973]
 [28.35499954 57.76399994 15.05799961]
 [26.33300018 57.77600098 18.23900032]
 [25.08600044 54.26399994 18.40200043]]
[[10.22099972 53.05599976 17.74900055]
 [10.58399963 55.20399857 14.63700008]
 [12.63899994 58.48400116 14.32499981]
 [10.53299999 61.81999969 14.44999981]]


```
int BCLoopSearch (const Coord *atoms1, int nr_atoms1, const Coord *atoms2, int nr_atoms2,  //flank1 and flank2
                  int looplength, //size of the gap/loop we are searching
                  int minloopmatch, int maxloopgap, //for partial matches: minimum total length, maximum gap
                  int mirror, //looking for mirrors?
                  float minBC, float maxR, //minimum BC score, maximum rigidity
                  const Coord *dbca, //CA database
                  int seg_index[][3], //(dbca offset, segment resnr, segment length)
                  int pdb_index[][2], int nr_pdbindex, //(seg_index offset, number of segments), total number of PDBs
                  int hits[][3], //pdbindex line, seg_index line, segment offset
                  float hitstats[][2] //score, rigidity
                 )
{
```

In [3]:
ctx.bcloopsearch = Transformer()
ctx.bcloopsearch.language = "c"
ctx.bcloopsearch.main_module.compiler_verbose = False
ctx.bcloopsearch.code.mount("bcloopsearch.c", authority="file")

ctx.bcloopsearch.main_module.lib.language = "c"
ctx.bclib_code >> ctx.bcloopsearch.main_module.lib.code 
ctx.bclib_code.mount("BCLoopSearch-lib.c", authority="file")

ctx.bc_hits = ctx.bcloopsearch

Execute local
Execute local


************************************************************
Exception in Seamless transformer: .bcloopsearch.tf.gen_header, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 236, in _execute_local
    raise Exception(msg)
Exception: Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/execute.py", line 79, in execute
    exec_code(code, identifier, namespace, inputs, output_name)
  File "/home/sjoerd/seamless/seamless/core/cached_compile.py", line 58, in exec_code
    exec(code_obj, namespace)
  File "Seamless transformer: .bcloopsearch.tf.gen_header", line 16, in <module>
    raise TypeError("Input schema needs to be defined")
TypeError: Input schema needs to be defined

************************************************************

************************************************************
Exception in Seamless transformer: .bcloopsearch.tf.gen_header, stream element None:
Traceback (

In [4]:
def set_example(bc):
    bc.atoms1 = np.zeros((4, 3))
    bc.atoms2 = np.zeros((4, 3))
    bc.looplength = 5
    bc.minloopmatch = 5
    bc.maxloopgap = 0
    bc.mirror = False
    bc.minBC = 0.9
    bc.maxR = 9999
    bc.dbca = np.zeros((10, 3))
    bc.seg_index = np.zeros((10,3), dtype=np.uint32)
    bc.pdb_index = np.zeros((10,2), dtype=np.uint32)

set_example(ctx.bcloopsearch.example)

schema = ctx.bcloopsearch.schema
schema.atoms1.form.contiguous = True
schema.atoms1.form.shape = (-1, 3)
schema.atoms2.form.contiguous = True
schema.atoms2.form.shape = (-1, 3)
schema.dbca.form.shape = (-1, 3)
schema.dbca.form.contiguous = True
schema.seg_index.form.shape = (-1, 3)
schema.seg_index.form.contiguous = True
schema.pdb_index.form.shape = (-1, 2)
schema.pdb_index.form.contiguous = True

MAXHITS = 100000
bcrx = ctx.bcloopsearch.result.example
bcrx.nhits = 0
bcrx.hits = np.zeros((MAXHITS,3), dtype=np.uint32)
bcrx.schema.hits.form.shape = (MAXHITS, 3)
bcrx.hitstats = np.zeros((MAXHITS,2), dtype=np.float32)
bcrx.schema.hitstats.form.shape = (MAXHITS, 2)

ctx.equilibrate()

Execute local


************************************************************
Exception in Seamless transformer: .bcloopsearch.tf.gen_header, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 236, in _execute_local
    raise Exception(msg)
Exception: Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/execute.py", line 79, in execute
    exec_code(code, identifier, namespace, inputs, output_name)
  File "/home/sjoerd/seamless/seamless/core/cached_compile.py", line 58, in exec_code
    exec(code_obj, namespace)
  File "Seamless transformer: .bcloopsearch.tf.gen_header", line 16, in <module>
    raise TypeError("Input schema needs to be defined")
TypeError: Input schema needs to be defined

************************************************************



Execute local
Waiting for: [Seamless transformer: .bcloopsearch.tf.integrator]
Execute local


set()

In [5]:
ctx.equilibrate()
set_example(ctx.bcloopsearch)
ctx.bcloopsearch.atoms1 = ctx.flank1
ctx.bcloopsearch.atoms2 = ctx.flank2
ctx.bcloopsearch.dbca = ctx.dbca
ctx.bcloopsearch.seg_index = np.array([[0,1,len(ctx.dbca.value)]],dtype=np.uint32)
ctx.bcloopsearch.pdb_index = np.array([[0,1]],dtype=np.uint32)
ctx.bcloopsearch.looplength = 7
ctx.bcloopsearch.minBC = 0
ctx.equilibrate()

Execute local
Execute local
gcc -c lib.c -O3 -ffast-math -march=native -fPIC -p -g -o lib.o

gcc -c main.c -O3 -ffast-math -march=native -fPIC -p -g -o main.o

Execute local


************************************************************
Exception in Seamless transformer: .bcloopsearch.tf.translator, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 236, in _execute_local
    raise Exception(msg)
Exception: Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/execute.py", line 77, in execute
    exec_code(code, identifier, namespace, inputs, output_name)
  File "/home/sjoerd/seamless/seamless/core/cached_compile.py", line 58, in exec_code
    exec(code_obj, namespace)
  File "Seamless transformer: .bcloopsearch.tf.translator", line 150, in <module>
    value = kwargs[prop]
KeyError: 'atoms2'

************************************************************



Execute local


************************************************************
Exception in Seamless transformer: .bcloopsearch.tf.translator, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 236, in _execute_local
    raise Exception(msg)
Exception: Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/execute.py", line 77, in execute
    exec_code(code, identifier, namespace, inputs, output_name)
  File "/home/sjoerd/seamless/seamless/core/cached_compile.py", line 58, in exec_code
    exec(code_obj, namespace)
  File "Seamless transformer: .bcloopsearch.tf.translator", line 150, in <module>
    value = kwargs[prop]
KeyError: 'dbca'

************************************************************



Execute local
Execute local


...

Execute local


************************************************************
Exception in <Unknown transformer>, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 246, in _execute_local
    manager.set_transformer_result(self.level1, self.level2, result, None, prelim=False)
  File "/home/sjoerd/seamless/seamless/core/manager.py", line 474, in set_transformer_result
    checksum, buffer = protocol.calc_buffer(value)
  File "/home/sjoerd/seamless/seamless/core/protocol/evaluate.py", line 73, in calc_buffer
    return get_hash(stream), stream
  File "/home/sjoerd/seamless/seamless/get_hash.py", line 8, in get_hash
    raise TypeError(type(content))
TypeError: <class 'bytearray'>
************************************************************

BCLoopSearch C function runs!!
NHITS 0


Execute local


************************************************************
Exception in <Unknown transformer>, stream element None:
Traceback (most recent call last):
  File "/home/sjoerd/seamless/seamless/core/jobscheduler.py", line 246, in _execute_local
    manager.set_transformer_result(self.level1, self.level2, result, None, prelim=False)
  File "/home/sjoerd/seamless/seamless/core/manager.py", line 474, in set_transformer_result
    checksum, buffer = protocol.calc_buffer(value)
  File "/home/sjoerd/seamless/seamless/core/protocol/evaluate.py", line 73, in calc_buffer
    return get_hash(stream), stream
  File "/home/sjoerd/seamless/seamless/get_hash.py", line 8, in get_hash
    raise TypeError(type(content))
TypeError: <class 'bytearray'>
************************************************************

BCLoopSearch C function runs!!
NHITS 1


set()

In [6]:
ctx.equilibrate()
print(ctx.bcloopsearch.schema)
print(ctx.bcloopsearch.result.schema)
print(ctx.bcloopsearch.header.value)
ctx.bcloopsearch.status

{'properties': {'atoms1': {'form': {'contiguous': True,
                                    'ndim': 2,
                                    'shape': [-1, 3]},
                           'items': {'form': {'bytesize': 8, 'type': 'number'}},
                           'storage': 'binary',
                           'type': 'array'},
                'atoms2': {'form': {'contiguous': True,
                                    'ndim': 2,
                                    'shape': [-1, 3]},
                           'items': {'form': {'bytesize': 8, 'type': 'number'}},
                           'storage': 'binary',
                           'type': 'array'},
                'dbca': {'form': {'contiguous': True,
                                  'ndim': 2,
                                  'shape': [-1, 3]},
                         'items': {'form': {'bytesize': 8, 'type': 'number'}},
                         'storage': 'binary',
                         'type': 'array'},
                

'OK'

In [7]:
ctx.bcloopsearch.inp.schema.mount("bcloopsearch-schema.json")
ctx.equilibrate()

set()

In [8]:
ctx.bc_hits.value

{'hits': array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint32), 'hitstats': array([[0.9981744 , 0.41393602],
       [0.        , 0.        ],
       [0.        , 0.        ],
       ...,
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]], dtype=float32), 'nhits': 1}

In [9]:
nhits = ctx.bc_hits.value.nhits.value
print(nhits)

1


In [10]:
ctx.bc_hits.value.data.value["hitstats"][:nhits]

array([[0.9981744 , 0.41393602]], dtype=float32)